# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
import json
import pathlib
import urllib.parse
import os
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
import psycopg2
import geodatasets
import warnings

from matplotlib.lines import Line2D
from matplotlib.widgets import RangeSlider
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

/Users/mac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'geodatasets'

In [ ]:
# Where data files will be read from/written to
DATA_DIR = pathlib.Path("./data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "8ITaLVGKJEzelLCfrNyuIi2rJ"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"
NYC_DATA_nypd='uip8-fykc.geojson'

DB_NAME = "project_database_final1"
DB_USER = "qianzhuoxin"
DB_URL = f"postgresql://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [ ]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():

    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

#### This section shows how we successfully load the five data files, we cleaned all the dataframes to clean, normalize and ensure the correct and consistent data types and dataframe types. We selected the columns based on the queries included in Part 3 and 4.

#### Downloads a GeoJSON file from a specified URL and saves it with a given filename

In [2]:
def download_nyc_geojson_data(url: str, jsonname: str,force: bool = False) -> str:
    """
    Arguments: 
        url: URL from which to download the GeoJSON data.
        jsonname: filename where the downloaded data will be saved.
        force: if set to True, the file will be downloaded again no matter whether it's existed.
               False as default
    
    Returns:
        the name of the file where the downloaded data is saved.
    """
    
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / (url_path.split('/')[-1])
    
    if force or not filename.exists():
        print(f"Downloading {url} to {jsonname}...")
        response = requests.get(url)
        response.raise_for_status()
       
        with open(jsonname, "a") as f:
            json.dump(response.json(), f)
        print(f"Done downloading {url}.")
    else:
        print(f"Reading from {jsonname}...")

    return jsonname

In [4]:
def clean_311_data(df: pd.DataFrame) -> gpd.GeoDataFrame:
    """
    Argument:
        pandas dataframe
    
    Return: 
        A GeoDataFrame with cleaned and transformed 311 service request data
    """
    columns = ['unique_key', 'created_date', 'incident_zip', 'complaint_type', 'longitude', 'latitude']
    df_selected = df[columns]
    df_selected = df_selected.dropna(axis = 0)
    
    df_selected.rename(columns = {'incident_zip': 'zipcode'}, inplace=True)
    df_selected['zipcode'] = df_selected['zipcode'].astype(int)
    df_selected.rename(columns = {'unique_key': 'id'}, inplace=True)
    df_selected.rename(columns = {'created_date': 'date'}, inplace=True)
    df_selected['geometry'] = gpd.points_from_xy(df_selected['longitude'], df_selected['latitude'], crs = "EPSG:4326")
    df_selected = df_selected.drop(['latitude','longitude'],axis = 1)
    df_selected['date'] = pd.to_datetime(df_selected['date']).dt.strftime('%Y-%m-%d')
    df_selected['date'] = pd.to_datetime(df_selected['date']).dt.date
    
    return gpd.GeoDataFrame(df_selected,geometry = 'geometry')

In [5]:
def download_311_data(offset: int) -> pd.DataFrame:
    """
    Argument:
        offset: show where data retrieval should start

    Returns:
        a pandas dataframe containing the downloaded 311 service request data
    """
    base_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}?$$app_token={NYC_DATA_APP_TOKEN}"
    url = (
        f'{base_url}&$limit=1000000&$offset={offset}'
        f'&$where=created_date between "2018-01-01T12:00:00.000" '
        f'and "2023-09-30T12:00:00.000"'
    )
    download_nyc_geojson_data(
        url, jsonname=f'query_{offset/1000000+1}_data.geojson', force=False
    )
    print('begin to create csv')
    df = gpd.read_file(f'query_{offset/1000000+1}_data.geojson')
    print('finishing create csv')
    csv_filename = f'query_{offset/1000000+1}_data.csv'
    df.to_csv(csv_filename, index=False)
    return pd.read_csv(csv_filename)

In [6]:
def load_311_data(offset:int = 1000000) -> pd.DataFrame:
    """
    Arguments:
        offset: show where data retrieval should start

    Returns:
        a pandas dataframe containing the downloaded 311 service request data
    """
    i = 1
    flag = True

    while flag:
        filename = f"query_{i}_data.csv"
        pathname = DATA_DIR / 'data311'/filename
        if pathname.exists():
            df = pd.read_csv(pathname)
        else:
            df = download_311_data(i*offset)
        if i == 1:
            df_311 = df
        else:
            df_311 = pd.concat([df_311,df])
        flag = df.shape[0]==offset
        i += 1
    return clean_311_data(df_311)

### zipcode data

In [7]:
def load_and_clean_zipcodes(ZIPCODE_DATA_FILE: pathlib.PosixPath) -> gpd.GeoDataFrame:
    """
    Argument:
        zipcode_datafile: The file path to the geospatial data file

    Returns:
        gpd.GeoDataFrame: a cleaned and processed GeoDataFrame
    """
    gdf = gpd.read_file(ZIPCODE_DATA_FILE)
    #normalized data
    gdf_normalized = gdf.to_crs(epsg=4326)
    
    #selected_zipcode
    columns=['ZIPCODE', 'PO_NAME', 'STATE','COUNTY','geometry']
    gdf_selected=gdf_normalized[columns]
    gdf_selected.rename(columns={'ZIPCODE': 'zipcode'}, inplace=True)
    gdf_selected.rename(columns={'COUNTY': 'county'}, inplace=True)
    gdf_selected.rename(columns={'STATE': 'state'}, inplace=True)
    gdf_selected.rename(columns={'PO_NAME': 'poname'}, inplace=True)
    
    #cleaned_zipcode
    gdf_cleaned = gdf_selected.dropna()
    gdf_cleaned.insert(0,"id", gdf_cleaned.index)
    return gdf_cleaned

### tree data

In [8]:
def download_tree_data():
    """
    Returns:
        a pandas DataFrame containing the tree data downloaded from the NYC Open Data source.
    """
    base_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}?$$app_token={NYC_DATA_APP_TOKEN}"
    url = f'{base_url}&$limit=1000000'
    download_nyc_geojson_data(url,jsonname = 'nyc_tree.geojson',force = False)
    df = gpd.read_file('nyc_tree.geojson')
    df.to_csv('nyc_tree.csv',index = False)
    
    df_read = pd.read_csv("nyc_tree.csv")
    return df_read

In [9]:
def clean_tree_data(df: pd.DataFrame) -> gpd.GeoDataFrame: 
    """
    Argument:
        pandas dataframe
    
    Return: 
        A GeoDataFrame with cleaned and transformed tree data
    """
    columns = ['tree_id','zipcode','longitude', 'latitude','spc_common','health','status']
    df_selected = df[columns]
    df_selected.rename(columns = {'tree_id': 'id'}, inplace = True)
    df_selected['geometry'] = gpd.points_from_xy(df_selected['longitude'],df_selected['latitude'],crs = "EPSG:4326")
    
    df_drop = df_selected.drop(['latitude','longitude'],axis = 1)
    df_cleaned = df_drop.dropna()
    return gpd.GeoDataFrame(df_cleaned,geometry = 'geometry')

In [10]:
def load_tree_data() -> gpd.GeoDataFrame:
    """
    Returns:
        a geopandas dataframe containing the downloaded 311 service request data
    """
    filename = "nyc_tree.csv"
    pathname = DATA_DIR / filename
    if pathname.exists():
        df = pd.read_csv(pathname)
    else:
        df = download_tree_data()
    return clean_tree_data(df)

### zillow data

In [11]:
def load_and_clean_zillow_data(ZILLOW_DATA_FILE: pathlib.PosixPath) -> pd.DataFrame:
    """
    Argument:
        zipcode_datafile: The file path to the geospatial data file

    Returns:
        gpd.GeoDataFrame: a cleaned and processed GeoDataFrame
    """
    df = pd.read_csv(ZILLOW_DATA_FILE)
    # Only choose NY data
    filtered_df = df[df['City'] == 'New York']
    
    # Change data from wide to long
    df_melt = filtered_df.melt(id_vars = ['RegionName'],value_vars = ['2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31',
       '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31','2015-11-30',
       '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
       '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30',
       '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28',
       '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31',
       '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',
       '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31',
       '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31',
       '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31',
       '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31',
       '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31',
       '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
       '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
       '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
       '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30',
       '2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28',
       '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
       '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31',
       '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31',
       '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30'],var_name = 'date',value_name = 'rent')
    df_cleaned = df_melt.dropna()
    df_cleaned.rename(columns = {'RegionName': 'zipcode'}, inplace = True)
    df_cleaned['date'] = pd.to_datetime(df_cleaned['date']).dt.strftime('%Y-%m-%d')
    df_cleaned['date'] = pd.to_datetime(df_cleaned['date']).dt.date
    df_cleaned.insert(0, "id", df_cleaned.index)
    return df_cleaned

eatra-part: nypd data


In [13]:
def download_and_clean_nypd_data() -> gpd.GeoDataFrame:
    """
    Returns:
        gpd.GeoDataFrame: A cleaned GeoDataFrame containing NYPD arrest data
    """
    base_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_nypd}?$$app_token={NYC_DATA_APP_TOKEN}"
    url = f'{base_url}&$limit=100000'
    download_nyc_geojson_data(url, jsonname = 'nyc_nypd.geojson', force = False)
    df = gpd.read_file(DATA_DIR / 'nyc_nypd.geojson')
    columns = ['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'age_group', 'geometry']
    df_selected = df[columns]
    df_cleaned = df_selected.dropna()
    return gpd.GeoDataFrame(df_cleaned, geometry = 'geometry')

### load all data

In [15]:
def load_all_data():
    """
    Returns:
        A tuple containing GeoDataFrames for all five data
    """
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = load_311_data(offset = 1000000)
    geodf_tree_data = load_tree_data()
    geodf_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    geodf_nypd_data = download_and_clean_nypd_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        geodf_zillow_data,
        geodf_nypd_data
    )

In [16]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, geodf_zillow_data, geodf_nypd_data = load_all_data()
warnings.filterwarnings("ignore")

NameError: name 'ZIPCODE_DATA_FILE' is not defined

### Get the data info and an outlook of what each datasets look like

In [18]:
geodf_zipcode_data.info()

NameError: name 'geodf_zipcode_data' is not defined

In [19]:

geodf_zipcode_data.head()

NameError: name 'geodf_zipcode_data' is not defined

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
geodf_zillow_data.info()

In [ ]:
geodf_zillow_data.head()

In [ ]:
geodf_nypd_data.info()

In [ ]:
geodf_nypd_data.head()

## Part 2: Storing Data

### Creating databese

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
connection = psycopg2.connect(dbname="postgres",user=DB_USER,host="localhost",port="5432" )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = connection.cursor()
cursor.execute("CREATE DATABASE {}".format(DB_NAME))
print(f"create {DB_NAME} sucessfully!")

### Creating extension

In [ ]:
connection1 = psycopg2.connect(dbname=DB_NAME,user=DB_USER,host="localhost",port="5432" )
cursor1 = connection1.cursor()
cursor1.execute('CREATE EXTENSION postgis;')

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_zipcode(
    zipcode INTEGER ,
    poname VARCHAR,
    state VARCHAR,
    county VARCHAR,
    geometry geometry(Polygon, 4326)
);

"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_311(
    id SERIAL PRIMARY KEY,
    date DATE,
    zipcode INTEGER,
    complaint_type VARCHAR,
    geometry GEOMETRY(Point, 4326)
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_tree(
    id SERIAL PRIMARY KEY,
    zipcode INTEGER,
    spc_common VARCHAR,
    health VARCHAR,
    status VARCHAR,
    geometry GEOMETRY(Point, 4326)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS zillow(
    zipcode INTEGER,
    date DATE,
    rent DECIMAL
);
"""

indance='''
CREATE INDEX idx_geometries_geom_gist ON zipcode USING gist (geometry)
'''

In [ ]:
# create that required schema.sql file
with open("schema.sql", "w") as f:
    f.write(ZIPCODE_SCHEMA)
   
    f.write(NYC_311_SCHEMA)
  
    f.write(NYC_TREE_SCHEMA)

    f.write(ZILLOW_SCHEMA)
    
    f.write(indance)

In [ ]:
with open("schema.sql", 'r') as file:
    sql_commands = file.read()
commands = sql_commands.split(';')

for command in commands:
    if command.strip(): 
        print(command)
        connection2 = psycopg2.connect(dbname=DB_NAME,user=DB_USER,host="localhost",port="5432" )
        cursor2 = connection2.cursor()
        cursor2.execute(command)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

In [ ]:
def write_dataframes_to_table(DB_USER,DB_NAME):
    engine = create_engine(f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}")
    geodf_zipcode_data.to_postgis('zipcode', engine, if_exists='append', index=False)
    geodf_311_data.to_postgis('nyc_311', engine, if_exists='append', index=False)
    geodf_tree_data.to_postgis('nyc_tree', engine, if_exists='append', index=False)
    geodf_zillow_data.to_sql('zillow', engine, if_exists='append', index=False)
    
write_dataframes_to_table(DB_USER,DB_NAME)

## Part 3: Understanding the Data

### Query 1

Query 1: Which area might be more calm to live in?

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(QUERY_1, QUERY_1_FILENAME):
    with open (QUERY_1_FILENAME,'w')as f:
        f.write(QUERY_1)
        f.close

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "query1"

QUERY_1 = """


"""
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

### Query 2

Query 2: Where has the most greenery?
Zipcode
tree_id
Using just the trees table, which 10 zip codes have the most trees?

The query result should have two columns, 10 rows. The rows should be sorted by the total number of trees, descending.


In [ ]:
QUERY_2_FILENAME = QUERY_DIR / "query2"

QUERY_2 = """

"""
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_2))
    for row in result:
        print(row)

### Query 3


Query 3: Can I afford a place in the areas with the most trees?
Zillow_rent
RegionName, 2023-08-31


Of the 10 zip codes with the most trees, for the month of August 2023, what is the average rent by zip code?

The query should have a JOIN statement. The query result should have two columns (not three) and 10 rows. The rows should be sorted by the total number of trees, descending. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128. 



In [ ]:
QUERY_3_FILENAME = QUERY_DIR / "query3"

QUERY_3 = """

"""
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_3))
    for row in result:
        print(row)

### Query4
Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?
2023-1
For the month of January 2023, return the 5 zip codes with the lowest average rent, and 5 zipcodes of the highest average rent, and include the tree count and complaint count for each zip code by using JOIN statements.

The query result should have 4 columns (zip code, average rent, tree count, and complaint count) and 10 rows: five with the highest average rent, and five with the lowest average rent. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128.


In [ ]:
QUERY_4_FILENAME = QUERY_DIR / "query4"

QUERY_4 = """

"""
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_4))
    for row in result:
        print(row)

### Query 5

Query 5: Where has the most greenery (take 2)?
Rewrite Query 2 to use both the trees table and the zipcodes table. Join both tables where the coordinate point of the tree is inside the polygon boundary of the zipcode as defined in the zipcode table.



The query should have a JOIN statement. The query results should match exactly the results of Query 2.



In [ ]:
QUERY_5_FILENAME = QUERY_DIR / "query5"

QUERY_5 = """


"""
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_5))
    for row in result:
        print(row)

### Query 6

Using the following coordinate pair on campus, which trees are within ½ mile radius of this
point?
Latitude: 40.80737875669467, Longitude: -73.96253174434912
Tree: longitude, latitude, tree_id, spc_common, health, status, x_sp, y_sp
The result should have 5 columns (ID, species, health, status, and coordinate location of each
tree).

In [ ]:
QUERY_6_FILENAME = QUERY_DIR / "query6"

QUERY_6 = """

"""
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_6))
    for row in result:
        print(row)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)